# Von Maximilian Horster und Yanik Plutte

# 1.0 Libaries

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import transforms, datasets
from torch.autograd import Variable

import numpy as np

import os

import matplotlib.pyplot as plt

# 2.0 Generall

In [2]:
print('Cuda is available:',torch.cuda.is_available(),'\n')

Cuda is available: False 



# 3.0 Hyperparameters

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

input_size = (8*4*4)
hidden_dim = 128

batch_size = 8

num_epochs = 8
learn_rate = 0.0003


# 4.0 Load Data

In [4]:
# create folder structure if it does not exist
folder_path = "../../data/"
if not os.path.exists(folder_path):
    os.makedirs(folder_path)


#MNIST Dataset
    #Create Folder, transorm to Tensor
train_dataset = torchvision.datasets.MNIST(
    root=folder_path, train=True, transform=transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.01307),(0.3081))]), download=True
)

test_dataset = torchvision.datasets.MNIST(
    root=folder_path, train=False, transform=transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.01307),(0.3081))])
)

#Data Loader
train_loader = torch.utils.data.DataLoader(
    dataset=train_dataset, batch_size=batch_size, shuffle=True #use shuffle to not let the network train on the order of data
)

test_loader = torch.utils.data.DataLoader(
    dataset=test_dataset, batch_size=batch_size, shuffle=True
)

#show MNIST Dataset

# figure= plt.figure(figsize=(10,8))
# spalten, zeilen = 5,5
# for i in range(1,spalten*zeilen+1):
#     sample_idx = torch.randint(len(train_dataset),size=(1,)).item()
#     image , label = train_dataset[sample_idx]
#     figure.add_subplot(spalten,zeilen,i)
#     plt.title(label, weight='bold')
#     plt.imshow(image.squeeze(), cmap='gray')
# plt.subplots_adjust(wspace=0.3,hspace=0.6)
# plt.show()


# 5.0 Classes

In [5]:
# Load necessary models for classification
    # reduce complexitiy and mass of code

import sys

sys.path.append('../src/')

from classes.autoencoder.autoencoderCNN import AutoEncoder
from classes.imageclassification.classification import MNIST_Classification_Class

## 5.1 Load AE Model

In [6]:
#Define AutoEncoder Model and use encoder

ModelAE = AutoEncoder().to(device)

ModelAE.eval()

trained_encoder = ModelAE.encode


## 5.2 Load Classification Model

In [7]:
#define Image classification model 

Model_Classifier = MNIST_Classification_Class(input_size, hidden_dim).to(device)

#  6.0 Function: View Predictions

In [8]:
def view_classifications(img,pred):
    pred = pred.cpu().data.numpy().squeeze()

    fig, (ax1, ax2, ax3) = plt.subplots(figsize=(6,9), ncols=2) 
    ax1.imshow(img.resize_(1, 28, 28).cpu().numpy().squeeze())
    ax1.set_title('Original MNIST Image')
    
    bars = ax2.barh(np.arange(10), pred)
    ax2.set_aspect(0.1)
    ax2.set_yticks(np.arange(10))               #Create bar graph to display prediction probability
    ax2.set_yticklabels(np.arange(10))
    ax2.set_title('Class Probability')
    ax2.set_xlim(0, 1.1)

    for bar, val in zip(bars, pred):
        if not (val < 0.3):
            ax2.text(val, bar.get_y() + bar.get_height() / 2, round(val, 2), va='center')

    plt.tight_layout()
    plt.show()
    plt.close()

# 7.0 Training

In [9]:
#Load Model / Continue Training
continue_training = True

PATH = "classification_model.pth"

# if continue_training:
#     ModelClassification = torch.load(PATH).to(device)

#criterion / lf
criterion = nn.CrossEntropyLoss()

#optimizer
optimizer = optim.Adam(params=Model_Classifier.parameters(),lr=learn_rate)

for epoch in range(num_epochs):
    for batch_id, (Bild,Label) in enumerate(train_loader):
        Bild = Bild.to(device)
        Label = Label.to(device)

        OutputEncoder = trained_encoder(Bild)
        OutputEncoder = OutputEncoder.to(device)
        OutputEncoder = OutputEncoder.view(OutputEncoder.size(0),-1)

        output = Model_Classifier(OutputEncoder)

        loss = criterion(output,Label)

        if batch_id %512 == 0:
            print('Image:',batch_id+1,'of',60000/batch_size,'in Epoche:',epoch+1,'// Loss: %.5f ' % loss)
            # view_classifications(Bild[0].cpu().view(1,28,28),output[0])
        

        optimizer.zero_grad()
        loss.backward()
        
        optimizer.step()
        
    print('Epoche:',epoch+1, 'Loss: %.5f ' % loss)

Image: 1 of 7500.0 in Epoche: 1 // Loss: 2.29299 
Image: 513 of 7500.0 in Epoche: 1 // Loss: 2.12798 
Image: 1025 of 7500.0 in Epoche: 1 // Loss: 2.14402 
Image: 1537 of 7500.0 in Epoche: 1 // Loss: 1.86320 
Image: 2049 of 7500.0 in Epoche: 1 // Loss: 1.93002 
Image: 2561 of 7500.0 in Epoche: 1 // Loss: 1.88088 
Image: 3073 of 7500.0 in Epoche: 1 // Loss: 1.70420 
Image: 3585 of 7500.0 in Epoche: 1 // Loss: 1.67653 
Image: 4097 of 7500.0 in Epoche: 1 // Loss: 1.74796 
Image: 4609 of 7500.0 in Epoche: 1 // Loss: 1.72289 
Image: 5121 of 7500.0 in Epoche: 1 // Loss: 1.71482 
Image: 5633 of 7500.0 in Epoche: 1 // Loss: 1.82349 
Image: 6145 of 7500.0 in Epoche: 1 // Loss: 1.93902 
Image: 6657 of 7500.0 in Epoche: 1 // Loss: 1.96168 
Image: 7169 of 7500.0 in Epoche: 1 // Loss: 1.67108 
Epoche: 1 Loss: 1.77742 
Image: 1 of 7500.0 in Epoche: 2 // Loss: 1.78798 
Image: 513 of 7500.0 in Epoche: 2 // Loss: 1.66863 
Image: 1025 of 7500.0 in Epoche: 2 // Loss: 1.64904 
Image: 1537 of 7500.0 in Epoc

# 8.0 Save Model Neural Network

In [10]:
torch.save(Model_Classifier,PATH)

# Confusuion Matrix

In [11]:
# label_list = torch.zeros(batch_size)
# pred_list = torch.zeros(batch_size)

# print(label_list, pred_list)

# with torch.no_grad():
#     for data,target in test_dataset:
#         data = data.reshape(-1,28,28).to(device)
#         target = target.to(device)

#         outconf= model1(data)

        

# Evaluation

In [12]:
# model1 = torch.load(FILE)

# with torch.no_grad():
#     for batch_id, (data,target) in enumerate(test_loader):
#         data = data.reshape(-1,28*28).to(device)  #[100, 1,28,28] > [100,734] ?
#         target = target.to(device)
        
#         output = model1(data) #was macht out?
#         loss = criterion(output,target) #hier out?!

#         view_classifications(data[0].view(1,28,28),output[0])
#         print('Batch in Epoche ',batch_id+1, 'Loss: %.5f ' % loss)

#     print('Epoche:',epoch+1, 'Loss: %.5f ' % loss)
